In [3]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

#deal with multi-labeled target:
from sklearn.multioutput import MultiOutputClassifier
import joblib

In [6]:
df = pd.read_csv('train.csv.zip')
text = df['comment_text']
targets = df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]

In [12]:
%%time
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)
word_vectorizer.fit(text)
train_word_features = word_vectorizer.transform(text)

CPU times: user 7.79 s, sys: 170 ms, total: 7.96 s
Wall time: 8.03 s


TfidfVectorizer(max_features=10000, stop_words='english',
                strip_accents='unicode', sublinear_tf=True,
                token_pattern='\\w{1,}')

In [97]:
%%time
clf = OneVsRestClassifier(LogisticRegression(C=0.1, solver='sag'), n_jobs=-1)
clf.fit(train_word_features, targets)

CPU times: user 455 ms, sys: 350 ms, total: 805 ms
Wall time: 3.95 s


OneVsRestClassifier(estimator=LogisticRegression(C=0.1, solver='sag'),
                    n_jobs=-1)

In [44]:
%%time
cv_score = np.mean(cross_val_score(clf, train_word_features, targets, cv=3, scoring='roc_auc'))
cv_score

0.9729936475742355

In [94]:
test_obj = "COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK"
vectorized_test_obj = word_vectorizer.transform([test_obj])
clf_2.predict_proba(vectorized_test_obj)

array([[0.40150184, 0.01659353, 0.1879106 , 0.00351474, 0.16640888,
        0.00943799]])

In [104]:
clf_2.predict_proba(vectorized_test_obj)[0]

array([0.05728791, 0.00578593, 0.02447301, 0.00266317, 0.02483698,
       0.00710184])

In [79]:
test_obj = """well you are such a beautiful human being!
i wanna be like you, rian, i love you so much I can't stand it.
i have a little feeling that the more I write here the more score I would get
so a little more words words words words and some good too
"""
vectorized_test_obj = word_vectorizer.transform([test_obj])
clf_2.predict_proba(vectorized_test_obj)

array([[0.18955007, 0.0081356 , 0.07431976, 0.00313142, 0.07606344,
        0.01205538]])

In [92]:
test_obj = "FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!"
vectorized_test_obj = word_vectorizer.transform([test_obj])
clf_2.predict_proba(vectorized_test_obj)

array([[0.97419413, 0.20960702, 0.97272647, 0.01075532, 0.82677044,
        0.05880348]])

In [18]:
print('Total CV score is {}'.format(np.mean(scores)))

Total CV score is 0.9730020618113876


In [103]:
with open('toxic_vectorizer_dump.pkl', 'wb') as output_file:
    joblib.dump(word_vectorizer, output_file)

with open('toxic_model_dump.pkl', 'wb') as output_file:
    joblib.dump(clf, output_file)